<a href="https://colab.research.google.com/github/chrisbrighouse/Public/blob/main/ReadPDFandSummarise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install --upgrade pip
!pip install --no-cache-dir openai
!pip install pdfplumber
!pip install glob2
!pip install openai
!pip install pypdf2

#!conda install -c conda-forge openai
#!pipenv install openai

# Authenticate to Google cloud drive
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 30.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 232.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 186.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 248.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.3 MB/s eta 0:00:00
Mounted at /gdrive


In [3]:
import requests
import os
import pdfplumber
from time import time,sleep
import textwrap
import re
import glob2
import openai
from IPython.display import Audio
from google.colab import drive
import PyPDF2

# Define the path to the Google Drive top level
file_folder_path = '/gdrive/MyDrive/'

filepath="input.txt"
def open_file(filepath):
  print (f"opening file: {filepath}")
  with open(filepath, 'r', encoding='utf-8') as infile:
    return infile.read()

def output_file_replace (filepath, content):
  print (f"output file: {filepath}")
  with open(filepath, 'w', )as f:
    f. write(content)
    return f

#single file version
def convert_pdf2txt(src_file, dest_file):
  try:
    with pdfplumber.open(src_file) as pdf:
     # pdfReader = PyPDF2.PdfFileReader(pdf)
     # print(pdfReader.getPage(0))
      output = ''
      for page in pdf.pages:
        # Extract text from each page and append to output
        output += page.extract_text()

      # Write output to file in destination directory
      with open(dest_file, "w") as outfile:
            outfile.write(output)

  except pdfplumber.PDFSyntaxError as e:
    print(f"PDFSyntaxError: {file}")
  except Exception as e:
    print(f"{e}: {file}")

#folder convert version
def convert_pdf2txt_dir(src_dir, dest_dir):
    files = os.listdir(src_dir)
    files = [i for i in files if '.pdf' in i]
    for file in files:
        try:
            with pdfplumber.open(os.path.join(src_dir, file)) as pdf:
                output = ''
                for page in pdf.pages:
                    # Extract text from each page and append to output
                    output += page.extract_text()
                # Write output to file in destination directory
                with open(os.path.join(dest_dir, f"{file}.txt"), "w") as outfile:
                    outfile.write(output)
        except pdfplumber.PDFSyntaxError as e:
            print(f"PDFSyntaxError: {file}")
        except Exception as e:
            print(f"{e}: {file}")



#use file name without extension in INPUT_NAME
INPUT_NAME="Diesel Generator Set Fuel System Preventative Maintenance"
#INPUT_NAME="AO-connector"
#INPUT_NAME="iso_50001_2017"
#INPUT_NAME="Decarbonizing the oil and"
#INPUT_NAME="product-data-sheet-rosemount-3051s-series"

SUMMARY_PROMPT_FILE="SummaryPrompt.txt"
OUTPUT_FILE=f"{INPUT_NAME}.txt"
OUTPUT_VOICE_FILE=f"{INPUT_NAME}.wav"

elapikey = open_file (file_folder_path + "API Keys/elevenlabskey.txt").strip()
openai.api_key = open_file (file_folder_path + "API Keys/openaikey.txt").strip()

input_file_path = f"{file_folder_path}PDFs/{INPUT_NAME}.pdf"
print (f"Input file: {input_file_path}")

summary_prompt_file_path = f"{file_folder_path}Prompts/{SUMMARY_PROMPT_FILE}"
print (f"Summary Prompt file: {summary_prompt_file_path}")

txt_input_file_path = f"{file_folder_path}Output/{INPUT_NAME}.txt"
print (f"Txt input file: {txt_input_file_path}")

output_file_path = f"{file_folder_path}Output/{OUTPUT_FILE}"
print (f"Output file: {output_file_path}")

output_voice_path = f"{file_folder_path}Output/{OUTPUT_VOICE_FILE}"
print (f"Output file: {output_voice_path}")

#this uses chatgpt
def gpt_3 (messages):
  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": messages}]
  )
  text = response['choices'][0]['message']['content']
  return text



def gpt_4(message):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        prompt=message,
        max_tokens=2048,
        temperature=0.5,
        n=1,
        stop=None,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].text.strip() 
 
def txt2speech (message):
  print ("Elevenlabs input: ", message)
  elapikey = open_file (file_folder_path + "API Keys/elevenlabskey.txt").strip()
  
  #Elevenlabs
  preview = "21m00Tcm4TlvDq8ikWAM"
  Ellie = "MF3mGyEYCl7XYWbV9V6O"
  Domi = "AZnzlk1XvdvUeBnXmlld"
  Bella = "EXAVITQu4vr4xnSDxMaL"
  Antoni = "ErXwobaYiN019PkySvjV"
  Josh = "TxGEqnHWrfWFTfGW9XjX"
  Arnold = "VR6AewLTigWG4xSOukaG"
  Adam = "pNInz6obpgDQGcFmaJgB"
  Sam = "yoZ06aMxZJJ28mfd3POQ"

  voice_id = Domi
  elevenlabs_tts_url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
  print (elevenlabs_tts_url)
  output_file = output_voice_path

  #Request payload
  headers = {
      "xi-api-key": elapikey,
      "accept": "audio/mpeg",
      "Content-Type": "application/json"
  }

  data = {
      "text": message,
      "lang": "en_US",
  }

  #Send request to Elevenlabs TTS API
  response = requests.post(elevenlabs_tts_url, headers=headers, json=data)

  # Check response status
  if response.status_code == 200:
    #Write audio to file
    with open(output_voice_path, "wb") as f:
      f.write(response.content)

    #Display audio
    display(Audio(output_voice_path))
  else:
    print(f"Error: {response.status_code}")



if __name__ == '__main__':

  #Generate text from the pdf
  convert_pdf2txt(input_file_path, txt_input_file_path)

  #build the full prompt
  summary_prompt = open_file(summary_prompt_file_path).replace('<<INPUT>>', open_file(txt_input_file_path))
  print(f"Summary prompt: {summary_prompt}")
  
  #Run the prompt through ChatGPT and save the output
  completion = gpt_3(summary_prompt)
  print('\n\n', completion)
  output_file_replace (output_file_path, completion)

  #Convert the output to voice audio
  txt2speech (completion)










opening file: /gdrive/MyDrive/API Keys/elevenlabskey.txt
opening file: /gdrive/MyDrive/API Keys/openaikey.txt
Input file: /gdrive/MyDrive/PDFs/Diesel Generator Set Fuel System Preventative Maintenance.pdf
Summary Prompt file: /gdrive/MyDrive/Prompts/SummaryPrompt.txt
Txt input file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.txt
Output file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.txt
Output file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.wav
opening file: /gdrive/MyDrive/Prompts/SummaryPrompt.txt
opening file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.txt
Summary prompt: Please summarise the following text in approximately 250 words:  
TM
Information Sheet # 19 Diesel Generator Set Fuel System
Your Reliable Guide for Power Solutions Preventative Maintenance
1.0 Introduction:
Fuel system maintenance is critical to the reliable perf